<a href="https://colab.research.google.com/github/jsvon/NLP-projects-/blob/main/Multi_Agent_Systems_using_Deepseek5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MULTI AGENT SYSTEMS

In [1]:
#Plotly for interactive visualization, geopandas for spatial data, Kaleido for static image eg PNG
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
import math
from typing import Optional, Tuple

from smolagents import tool, DuckDuckGoSearchTool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [7]:
#setting up the agent

import os
from PIL import Image
from smolagents.models import HfApiModel # Import HfApiModel from smolagents.models
from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool

# with inference peovider model = HfApiModel(model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud', provider="together")
model = HfApiModel(model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud')

In [8]:
# simple agent created first ro act as a baseline

task = """
     Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time.
"""

In [9]:
agent = CodeAgent(
    model = model,
    tools = [
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time
        ],
        additional_authorized_imports = ["pandas"],
        max_steps = 20,
)

In [10]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations_search = web_search(query="Batman filming locations")                                           
  print(batman_locations_search)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Where The Batman Filmed (All Gotham Real-Life Locations) - Screen 
Rant](https://screenrant.com/where-was-the-batman-movie-filmed/)
The London location known as Two Temple Place served as another The Batman filming location. Two Temple Place was 
the first building seen in the movie, in fact, as it was the home of Gotham's Mayor Don Mitchell. Matt Reeves began
The Batman by showing the location through Riddler's eyes. Two Temple Place then became a major location for the 
...

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom.Most of the recognizable buildings of this new version of Gotham 
City are located in Liverpool, although Chicago was a ...

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
The Batman location: the subterranean entrance to 'Wayne Tower': Holborn tram tunnel, Holborn It's accessed via a 
subterranean passage, which is the old Kingsway Tram Tunnel , disused since trams were discontinued in London in 
1952, smack in the middle of busy Southampton Row just south of Theobald's Road near Holborn tube station.

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
For The Batman, Matt Reeves mainly filmed in the UK and Chicago to capture the cold, gothic, and older feel of 
Gotham. While The Batman did try to film in real locations, iconic spots like the Batcave and Wayne Tower were 
built on studio sets. When filming on location, the cast and crew spent most of their time in the UK for the main 
Gotham scenes.

[12 Batman Movie Locations You Can Visit! - Leisure Asia 
Global](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Image: Courtesy Rob Farrow / Two, Temple Place, London/ Creative Commons Attribution-Share Alike 2.0 Generic/ 
Wikimedia Commons The stunning Victorian mansion in London is another officially IMDb listed filming location for 
Matt Reeves' The Batman.The building is as gorgeous as it can be and retains the classic European charm till date. 
The venue has been featured in several other noted ...

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-filmed/)
The Batman Filming Locations 'The Batman' was filmed in various parts of England and Scotland, specifically in 
Liverpool, London, Watford, Bedford, Glasgow, and Shotts. Some portions were also filmed in Illinois, particularly 
in Chicago. Principal photography for the movie commenced on January 27, 2020, but was abruptly halted on March 14 
...

[These Are the Cities Standing in for Gotham in 'The Batman' - 
Distractify](https://www.distractify.com/p/the-batman-filming-locations)
The latest film centered around the Caped Crusader, The Batman, is finally in theaters.One of the key elements of 
any good Batman story is the Gotham City setting. Over the years, various directors have chosen different cities to
stand in for the crime-riddled location, but where did Matt Reeves set his epic Batman tale?

[12 Batman Movie Locations You Can Actually Visit - AugustMan 
Singapore](https://www.augustman.com/sg/entertainment/travel/batman-movie-locations-you-can-visit/)
Image: Courtesy Superchilum, CC BY-SA 4.0, via Wikimedia Commons One of the many IMDb listed filming locations for 
Reeves' The Batman is the iconic St. George's Hall in Liverpool, England. Though we have seen different variations 
of Gotham City under Christopher Nolan and Tim Burton, Reeves

[Step 1: Duration 7.15 seconds| Input tokens: 2,378 | Output tokens: 177]

...

[Step 4: Duration 23.35 seconds| Input tokens: 31,459 | Output tokens: 1,325]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Define coordinates for each location                                                                           
  locations_coords = {                                                                                             
      "Two Temple Place, London": (51.5121, -0.0802),                                                              
      "Wellington's Column and Steble Fountain, Liverpool": (53.4084, -2.9927),                                    
      "St George's Hall, Liverpool": (53.4093, -2.9988),                                                           
      "Marshall Suloway Bridge, Chicago": (41.8913, -87.6338),                                                     
      "Enzo Ferrari Museum, Maranello, Italy": (44.6778, 10.4347),                                                 
      "Porsche Museum, Leipzig, Germany": (51.2634, 12.3913),                                                      
      "Lamborghini Museum, Sant'Agata Bolognese, Italy": (44.5007, 11.2108),                                       
      "McLaren Technology Centre, Woking, UK": (51.2253, -0.5960)                                                  
  }                                                                                                                
                                                                                                                   
  # Gotham City coordinates                                                                                        
  gotham_city_coords = (40.7128, -74.0060)                                                                         
                                                                                                                   
  # Function to calculate travel time for cargo plane                                                              
  def calculate_travel_time(coords1, coords2):                                                                     
      return calculate_cargo_travel_time(                                                                          
          origin_coords=coords1,                                                                                   
          destination_coords=coords2                                                                               
      )                                                                                                            
                                                                                                                   
  # Calculate travel times for all locations                                                                       
  travel_times = {loc: calculate_travel_time(coord, gotham_city_coords) for loc, coord in                          
  locations_coords.items()}                                                                                        
                                                                                                                   
  # Create a pandas DataFrame                                                                                      
  df = pd.DataFrame(list(travel_times.items()), columns=['Location', 'Travel Time (hours)'])                       
  final_answer(df)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:                                             Location  Travel Time (hours)
0                           Two Temple Place, London                 9.17
1  Wellington's Column and Steble Fountain, Liver...                 8.81
2                        St George's Hall, Liverpool                 8.81
3                   Marshall Suloway Bridge, Chicago                 2.68
4              Enzo Ferrari Museum, Maranello, Italy                10.67
5                   Porsche Museum, Leipzig, Germany                10.36
6    Lamborghini Museum, Sant'Agata Bolognese, Italy                10.76
7              McLaren Technology Centre, Woking, UK                 9.14

[Step 5: Duration 23.56 seconds| Input tokens: 49,350 | Output tokens: 2,055]

In [12]:
result

,Location,Travel Time (hours)
0,"Two Temple Place, London",9.17
1,"Wellington's Column and Steble Fountain, Liver...",8.81
2,"St George's Hall, Liverpool",8.81
3,"Marshall Suloway Bridge, Chicago",2.68
4,"Enzo Ferrari Museum, Maranello, Italy",10.67
5,"Porsche Museum, Leipzig, Germany",10.36
6,"Lamborghini Museum, Sant'Agata Bolognese, Italy",10.76
7,"McLaren Technology Centre, Woking, UK",9.14


In [13]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│                                                                                                                 │
│      Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here     │
│ (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                         │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Perform a web search to find a comprehensive list of Batman filming locations worldwide.
2. For each filming location, visit the webpage to confirm the location's details.
3. Use a geocoding service to find the coordinates of each Batman filming location.
4. Calculate the travel time for a cargo plane from each Batman filming location to Gotham (40.7128° N, 74.0060° 
W).
5. Perform a web search to find a comprehensive list of supercar factories worldwide.
6. For each supercar factory, visit the webpage to confirm the factory's details.
7. Use a geocoding service to find the coordinates of each supercar factory.
8. Calculate the travel time for a cargo plane from each supercar factory to Gotham (40.7128° N, 74.0060° W).
9. Compile the Batman filming locations and their travel times into a pandas dataframe.
10. Compile the supercar factories and their travel times into the same pandas dataframe.
11. Provide the final answer as a pandas dataframe.

<end_plan>
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for Batman filming locations                                                                            
  batman_filming_locations_queries = [                                                                             
      "list of batman filming locations",                                                                          
      "batman filming locations around the world",                                                                 
      "locations where batman was filmed"                                                                          
  ]                                                                                                                
                                                                                                                   
  # Perform searches                                                                                               
  batman_filming_locations_search_results = [web_search(query=query) for query in                                  
  batman_filming_locations_queries]                                                                                
                                                                                                                   
  # Print search results for review                                                                                
  for idx, result in enumerate(batman_filming_locations_search_results):                                           
      print(f"Search #{idx + 1} result: {result}\n")                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Search #1 result: ## Search Results

[Where The Batman Filmed (All Gotham Real-Life Locations) - Screen 
Rant](https://screenrant.com/where-was-the-batman-movie-filmed/)
Gotham's cemetery was brought to life using two The Batman filming locations. This included filming portions of 
scenes at the cemetery in Glasgow, Scotland. The crew traveled to the Necropolis Cemetery, a Victorian cemetery 
founded in 1832. The Batman used this location to film Batman and Catwoman's interaction at the end of the movie 
...

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom.Most of the recognizable buildings of this new version of Gotham 
City are located in Liverpool, although Chicago was a ...

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
The Batman location: the subterranean entrance to 'Wayne Tower': Holborn tram tunnel, Holborn It's accessed via a 
subterranean passage, which is the old Kingsway Tram Tunnel , disused since trams were discontinued in London in 
1952, smack in the middle of busy Southampton Row just south of Theobald's Road near Holborn tube station.

[Where Was The Batman Filmed? | iFILMthings](https://ifilmthings.com/where-was-the-batman-filmed/)
Full List of The Batman Filming Locations. The following list acquired from IMdbPro are all 37 filming locations 
for The Batman. Warner Bros. Studios Leavesden, Warner Drive, Leavesden, Watford, Hertfordshire, England, UK 
(Interior: Batcave / Wayne Tower / Selina's Apartment / Exterior: Iceberg Lounge)

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
Matt Reeves' The Batman was filmed in many places around the globe to create a new version of Gotham and used 
real-life locations to capture the right look. This new take on the Dark Knight introduced audiences to Robert 
Pattinson's Batman and the environment he lives in. Along with bringing characters like Penguin, Catwoman, and 
others from DC Comics to life, the movie truly made Gotham ...

[12 Batman movie locations around the world you can actually 
visit](https://www.lifestyleasia.com/sg/entertainment/the-batman-movie-locations-around-the-world-you-can-visit/)
Image: Courtesy Superchilum, CC BY-SA 4.0, via Wikimedia Commons One of the many IMDb listed filming locations for 
Reeves' The Batman is the iconic St. George's Hall in Liverpool, England. Though we have seen different variations 
of Gotham City under Christopher Nolan and Tim Burton, Reeves seems to have stuck to the comic books' description 
quite accurately and has retained the European ...

[The Batman Filming Locations: Where The Batcave Is Located? - 
OtakuKart](https://otakukart.com/the-batman-filming-locations-where-the-batcave-is-located/)
Marshall Suloway Bridge, situated in Chicago, became a part of the Filming locations of The Batman when we saw 
Bruce riding his motorcycle on the Bridge. Even Marshall Suloway Bridge is one of the historical locations in 
Chicago because it was built back in 1928 by a steel company investing $2,500,000. So, if you are in Chicago, do 
ride your ...

[The Batman Filming Locations (2022)](https://a2zfilminglocation.com/the-batman-filming-locations-2022/)
The filming of The Batman film started in January 2020 under the working title Vengeance and wrapped on March 21, 
2021. The second unit filming took place from 17 September 2020 to 13 March 2021. In the below article, we will see
the different locations where The Batman film was filmed. Here

...

[Step 9: Duration 171.05 seconds| Input tokens: 138,313 | Output tokens: 7,809]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
422 Client Error: Unprocessable Entity for url: 
https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud/v1/chat/completions (Request ID: TrEyRK)

Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32768. Given: 33044 `inputs` tokens and 0 
`max_new_tokens`
Make sure 'text-generation' task is supported by the model.

[Step 10: Duration 0.94 seconds| Input tokens: 166,118 | Output tokens: 9,561]

AgentGenerationError: Error in generating model output:
422 Client Error: Unprocessable Entity for url: https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud/v1/chat/completions (Request ID: TrEyRK)

Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32768. Given: 33044 `inputs` tokens and 0 `max_new_tokens`
Make sure 'text-generation' task is supported by the model.

In [ ]:
detailed_report

[{'Location': 'Ferrari, Maranello, Italy',
  'Coordinates': (44.596, 11.0736),
  'Distance to Gotham (km)': 6640.354662490861,
  'Cargo Plane Transfer Time (hours)': 8.019752007839205},
 {'Location': "Lamborghini, Sant'Agata Bolognese, Italy",
  'Coordinates': (44.549, 11.3665),
  'Distance to Gotham (km)': 6663.168992172412,
  'Cargo Plane Transfer Time (hours)': 8.047305546101947},
 {'Location': 'Koenigsegg, Ängelholm, Sweden',
  'Coordinates': (56.4491, 12.8478),
  'Distance to Gotham (km)': 6169.905716999168,
  'Cargo Plane Transfer Time (hours)': 7.451576952897546},
 {'Location': 'Porsche, Leipzig, Germany',
  'Coordinates': (51.3067, 12.3354),
  'Distance to Gotham (km)': 6378.323168079406,
  'Cargo Plane Transfer Time (hours)': 7.703288850337447},
 {'Location': 'McLaren, Woking, UK',
  'Coordinates': (51.3572, -0.6373),
  'Distance to Gotham (km)': 5541.900478379156,
  'Cargo Plane Transfer Time (hours)': 6.693116519781589},
 {'Location': 'Pagani, San Cesario sul Panaro, Italy',

In [ ]:
# improving system structure to avoid it being slower
#splitting the task between two agents
# Each agent will be more focused on its task
# Separating memories reduces count of input tokens

model = HfApiModel(model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud')

web_agent = CodeAgent(
    model = model,
    tools = [
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],

    name = "web_agent",
    description = "Browses the web to find information",
    verbosity_level = 0,
    max_steps =10,
)


In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [ ]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a duckduckgo web search  │ query (`string`): The search      │
            │                             │ based on your query (think a      │ query to perform.                 │
            │                             │ Google search) then returns the   │                                   │
            │                             │ top search results.               │                                   │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.                         │ destination_coords (`array`):     │
            │          

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

[Step 1: Duration 0.00 seconds]

ValueError: You must provide an api_key to work with Together API or log in with `huggingface-cli login`.